In [1]:
!make -C csrc/bcc/libbpf-tools

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'
make: Nothing to be done for 'all'.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bcc/libbpf-tools'


In [2]:
!make -C csrc/bpftool/src bpftool

make: Entering directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'
...                        libbfd: [ OFF ]
...               clang-bpf-co-re: [ on  ]
...                          llvm: [ on  ]
...                        libcap: [ OFF ]
make: 'bpftool' is up to date.
make: Leaving directory '/Users/szhong/Downloads/bpf-study/csrc/bpftool/src'


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import pandas as pd

from depsurf import BCC_OBJ_PATH, OUTPUT_PATH, BPFObject, gen_report, Versions
from depsurf.output import save_df
from collections import defaultdict

result_by_prog = {}
result_by_dep = defaultdict(dict)
dep_prog = defaultdict(list)
for path in sorted(BCC_OBJ_PATH.glob("*.bpf.o")):
    bpf = BPFObject(path)
    result_path = OUTPUT_PATH / "bcc" / f"{bpf.name}.log"
    report = gen_report(bpf.deps, [Versions.REGULAR, Versions.ARCH], result_path)
    for k, v in report.items():
        result_by_prog[(bpf.name, *k)] = v
        result_by_dep[k] |= v
        dep_prog[k].append(bpf.name)

for k, v in result_by_dep.items():
    v[("Program", "", "")] = dep_prog[k]


df = pd.DataFrame(result_by_prog).T
df.index.names = ["Prog", "Type", "Name"]
save_df(df, "bcc")

df = pd.DataFrame(result_by_dep).T
df.index.names = ["Type", "Name"]
save_df(df, "dep")

print("Done")

[       report.py:38 ] INFO: Report saved to /Users/szhong/Downloads/bpf-study/output/bcc/bashreadline.log
[  func_groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.4.0-21-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.4.0-21-generic-amd64.jsonl
[  func_groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.8.0-22-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.8.0-22-generic-amd64.jsonl
[  func_groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.10.0-19-generic-amd64.jsonl
[       symtab.py:70 ] INFO: Loading symtab from /Users/szhong/Downloads/bpf-study/data/symtab/4.10.0-19-generic-amd64.jsonl
[  func_groups.py:68 ] INFO: Loading funcs from /Users/szhong/Downloads/bpf-study/data/dwarf_funcs/4.13.0-16-generic-amd64.jsonl
[     

Done
